In [1]:
import pandas as pd
pd.set_option("display.max_columns", 100)
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Optional, Any, Union

IN_CSV_DATA = Path().cwd().parent.parent / "data/2_loaded"
OUT_CSV_DATA = Path().cwd().parent.parent / "data/3_feature_engineered"

In [2]:
# Load the summary  for reference later
df_summary = pd.read_csv(IN_CSV_DATA/'summary/ride_summary_good.csv')
df_summary['start_date'] = pd.to_datetime(df_summary['start_date'])
df_summary['year'] = df_summary['start_date'].dt.year
df_summary['month'] = df_summary['start_date'].dt.month
df_summary['month_name'] = df_summary['start_date'].dt.month_name()
df_summary['start_time'] = pd.to_timedelta(df_summary['start_time'])
df_summary['end_time'] = pd.to_timedelta(df_summary['end_time'])

## Handle categorical variables
month_order = {'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}
df_summary['month_name'] = pd.Categorical(df_summary['month_name'] , categories=month_order.keys(), ordered=True)

In [3]:
# Filter out years 2018, and 2025
df_summary = df_summary.loc[~df_summary['year'].isin([2018,2025]),:].reset_index(drop=True)

In [4]:
df_summary.columns

Index(['ride_id', 'start_date', 'start_time', 'end_time', 'biker_weight_lbs',
       'bike_weight', 'bag_weight', 'avg_speed', 'avg_cruising_speed',
       'total_ride_time_sec', 'total_moving_time_sec', 'total_distance_mi',
       'total_ascent_ft', 'total_descent_ft', 'avg_heart_rate', 'avg_power',
       'avg_cadence', 'avg_ambient_temp_F', 'best_power_4s', 'best_power_5s',
       'best_power_10s', 'best_power_20s', 'best_power_30s', 'best_power_1m',
       'best_power_2m', 'best_power_3m', 'best_power_4m', 'best_power_5m',
       'best_power_6m', 'best_power_10m', 'best_power_20m', 'best_power_30m',
       'best_power_40m', 'best_power_1h', 'best_power_2h', 'year', 'month',
       'month_name'],
      dtype='object')

In [5]:
cols_of_interest = ['ride_id','start_date','start_time','end_time','total_distance_mi','avg_speed','avg_cruising_speed',
                    'total_ride_time_sec','total_moving_time_sec','total_ascent_ft','total_descent_ft','year','month_name']
power_cols = ['best_power_4s', 'best_power_5s','best_power_10s', 'best_power_20s', 'best_power_30s', 'best_power_1m',
            'best_power_2m', 'best_power_3m', 'best_power_4m', 'best_power_5m','best_power_6m', 'best_power_10m', 
            'best_power_20m', 'best_power_30m','best_power_40m', 'best_power_1h', 'best_power_2h']
cols_of_interest += power_cols
df_summary = df_summary.loc[:, cols_of_interest]

for col in power_cols:
    df_summary[col] = df_summary[col].fillna(0.0) # for the power columns, fill the nulls with zeros

In [6]:
df_summary.head()

,ride_id,start_date,start_time,end_time,total_distance_mi,avg_speed,avg_cruising_speed,total_ride_time_sec,total_moving_time_sec,total_ascent_ft,total_descent_ft,year,month_name,best_power_4s,best_power_5s,best_power_10s,best_power_20s,best_power_30s,best_power_1m,best_power_2m,best_power_3m,best_power_4m,best_power_5m,best_power_6m,best_power_10m,best_power_20m,best_power_30m,best_power_40m,best_power_1h,best_power_2h
0,0x5ccb413c,2019-05-02,0 days 19:13:00,0 days 19:53:00,8.865922,13.784195,14.574504,2401.0,2143.0,300.5396,300.5396,2019,May,485.282271,481.991827,462.283537,412.144319,391.244346,314.816769,209.581472,186.631718,179.636006,157.569641,155.009664,142.368486,134.442850,130.264839,0.000000,0.0,0.0
1,0x5cccc63c,2019-05-03,0 days 22:52:44,0 days 23:53:54,9.516836,11.228317,12.081964,3671.0,2725.0,333.3496,280.1974,2019,May,432.549324,425.735809,389.502414,330.267900,299.166732,225.907473,143.793900,147.597113,129.794118,118.283785,115.819729,107.256435,98.405537,97.000798,94.003892,0.0,0.0
2,0x5cdbff9c,2019-05-15,0 days 12:01:32,0 days 12:37:36,8.029630,14.256866,14.701154,2165.0,1942.0,255.2618,211.2964,2019,May,422.177957,419.814683,393.709515,363.263188,336.841737,271.435198,215.164384,187.689124,188.538262,164.696769,157.267499,154.668660,139.481512,138.606140,0.000000,0.0,0.0
3,0x5cdc8391,2019-05-15,0 days 21:24:33,0 days 22:03:38,8.452656,13.534030,14.453954,2346.0,2044.0,234.9196,263.4643,2019,May,416.729572,414.718966,393.124198,327.432493,295.389433,231.954659,191.952644,182.999122,175.942510,169.921718,170.186598,156.173244,143.046556,137.236242,0.000000,0.0,0.0
4,0x5ce7dff1,2019-05-24,0 days 12:13:37,0 days 12:47:43,8.018602,14.740176,15.179620,2047.0,1880.0,260.1833,206.7030,2019,May,419.659872,414.650607,402.052528,376.548577,344.272237,262.329945,225.472466,193.519868,187.969739,181.348731,173.976554,167.162898,150.946458,151.020599,0.000000,0.0,0.0


# Making New Features
## 1. Time Since Last Ride
How long has it been since the last ride?|

In [7]:
df_summary['start_time'] = df_summary['start_date'] + df_summary['start_time']
df_summary['end_time'] = df_summary['start_date'] + df_summary['end_time']

In [8]:
# ensure that the data is sorted chronologically
df_summary = df_summary.set_index('start_time').sort_index().reset_index()

# create a temporary column for the Last end time in UTC seconds
df_summary['last_ride_end'] = df_summary['end_time'].apply(lambda x: x.timestamp()).shift()

sec2hour = 1.0/3600 # convert seconds to hours
df_summary['hours_since_last_ride'] = sec2hour*(df_summary['start_time'].apply(lambda x: x.timestamp()) - df_summary['last_ride_end'])
df_summary['hours_since_last_ride'] = df_summary['hours_since_last_ride'].fillna(10000) # assume any contribution to fatigue/fitness is 0 ..

df_summary.drop(['last_ride_end'], axis=1, inplace=True) # drop temporary column

## 2. Active Ride Time Ratio
What percentage of the ride was I resting vs moving?

In [9]:
df_summary['active_time_ratio'] = df_summary['total_moving_time_sec'] / df_summary['total_ride_time_sec']

## 3. Avg Climb Rate
How fast did the total elevation ascent occur in. This is proportional to climbing power required to change vertical potential energy in that same time frame.

In [10]:
df_summary['avg_climb_rate'] = df_summary['total_ascent_ft'] / (df_summary['total_moving_time_sec'] / 60.0) # units of feet per minute

## 4. Relative Distance Intensity within an 4 week Training Window 
Within a 4 week training window, what is the maximum miles ridden? 
Then for each ride, what is the ratio of the ride distance to the max distance in that training window? --> Intensity proxy

In [11]:
df_training_windows = pd.DataFrame(pd.date_range(start='01/01/2019', end='12/31/2024', freq='4W-SUN'), columns=['window_start_date'])
df_training_windows['window_number'] = range(df_training_windows.shape[0])

In [12]:
pd.merge_asof(df_summary, df_training_windows, left_on='start_date', right_on='window_start_date')\
    .loc[:,['ride_id','start_date','window_start_date','window_number']].head(10)

,ride_id,start_date,window_start_date,window_number
0,0x5ccb413c,2019-05-02,2019-04-28,4
1,0x5cccc63c,2019-05-03,2019-04-28,4
2,0x5cdbff9c,2019-05-15,2019-04-28,4
3,0x5cdc8391,2019-05-15,2019-04-28,4
4,0x5ce7dff1,2019-05-24,2019-04-28,4
5,0x5ce8606e,2019-05-24,2019-04-28,4
6,0x5ce95b96,2019-05-25,2019-04-28,4
7,0x5cf1ae78,2019-05-31,2019-05-26,5
8,0x5cf519b8,2019-06-03,2019-05-26,5
9,0x5cf59935,2019-06-03,2019-05-26,5


In [13]:
df_summary = pd.merge_asof(df_summary, df_training_windows, left_on='start_date', right_on='window_start_date')

In [14]:
df_summary['max_training_window_distance'] = df_summary.groupby('window_number')['total_distance_mi'].transform('max')

In [15]:
df_summary['distance_training_intensity'] = df_summary['total_distance_mi'] / df_summary['max_training_window_distance'] 

In [16]:
df_summary.drop(columns=['max_training_window_distance','window_start_date','window_number'], inplace=True)

## 5. Time-weighted Intensity (Training Load)
How intense was the last ride and how much of its residual effects are felt by the time the current ride is being performed.

Here we will assume that for any given ride, my muscles and their fatigue recovers in ~1 week of pure rest. 

So we can represent the time-weighted intensity as a decaying exponential with time constant $3*\tau\approx7$ _days_ which means $\tau\approx56$ hours.
This time constant implies that after 56 hours, my fatigue from 1 ride will have decreased by 63%.

(Semi-)Formally: $Load_{ride[k-1]}=Intensity_{ride[k-1]}*exp(\frac{-t_{since}}{\tau})$

----------------------------------------
### Calculating a Ride's Cumulative Training Load (up until start time)
To contextualize where $ride_{k}$ starts at, it is useful to sum up the training loads from each ride $0,1,...,k-1$ leading up to it. Most rides where the time since is more than $5\tau$ away will contribute nothing.

In [27]:
tau = 56 # hours, the time constant for training load intensity decay
df_summary['prior_training_load'] = 0.0

for k, row_k in df_summary.iterrows():
    if k==0: continue # skip the first row since it is guaranteed to be 0.0 load
    training_loads = []
    for j, row_j in df_summary.iterrows():
        if j==k: break # stop calculating the training loads
        intensity_j = row_j['distance_training_intensity']
        time_since_ride_k = row_k['start_time'].timestamp() - row_j['end_time'].timestamp()
        time_since_ride_k = time_since_ride_k * sec2hour
        load = intensity_j * np.exp(-time_since_ride_k / tau)
        training_loads.append(load)
    df_summary.loc[k,'prior_training_load'] = np.sum(training_loads)